# Validata Generate Channel RM Stage

### Data Generation

We had simulated data using Oskar. The simulation scripts are present in `scripts/ska_low_sim`. (refer [confluence page](https://confluence.skatelescope.org/display/SE/DHR-311%3A+Script+to+simulate+SKA-LOW+visibilities))

#### Simulation Configuration

For simulation following configuration is used. (further refered as `custom_sim.yaml`)

``` yaml
scenario: "low40s-model"         

# ===============================
# Global simulation parameters
# ===============================

n_stations: 40                                         # Number of stations
tel_model: "./telescope-models/SKA-Low_AA2_40S_rigid-rotation_model.tm" # Telescope model directory

simulation_start_frequency_hz: 123.0e6                  # Start frequency (Hz)
simulation_end_frequency_hz: 153.0e6                    # End frequency (Hz)
correlated_channel_bandwidth_hz: 21.70138888888889e3    # Channel width (Hz)

observing_time_mins: 10                              # Observation duration (minutes)
sampling_time_sec: 3.3973862400000003                   # Dump/integration time (seconds)

fields:
  EoR2:
    Cal1:
      ra_deg: 197.914612
      dec_deg: -22.277973
      scan_id_start: 300
      transit_time: "2000-01-03 22:33:30.000"

# ===============================
# Options for run_sim.py
# ===============================

run_sim:
  oskar_sif: "./OSKAR-2.11.1-Python3.sif" # Path to OSKAR Singularity image

  # GLEAM sky model.
  gleam_file: "./sky-models/GLEAM_EGC.fits" # GLEAM catalogue FITS file
  field_radius_deg: 10.0            # Radius of field of view (degrees)

  # Corruptions to be applied
  tec_screen: "./tec/calibrator_iono_tec.fits" # Ionospheric TEC screen FITS

  # Imaging parameters using wsclean. Optional. Comment to disable.
  create_dirty_image: true          # Whether to run wsclean imaging
  image_size: 1024                  # Image size (pixels)
  pixel_size: "2arcsec"             # Pixel size (angular units)

  # Extra parameters to pass directly to run_oskar.py
  run_oskar_extra_params: "--use-gpus --double-precision"
```

Follow steps mentioned in confluence page for data and enviornment setup. Run the following command to simulate visibilities. 
`python run_sim.py custom_sim.yaml`

This simulation has ionospheric and channel rotation measure effects. With the above config simulation script images the simulated visibilities, these images can be used for further verification. 


In [2]:
## Setup

input_data = "/home/ska/Work/data-simulation/low40s-model-channel-rm-281025_075520/visibility.scan-300.ms"
cache = "./output_6gb"
dask_schduler = "10.131.131.55:8786"
pipeline_output_path = "./output_6gb"
over_sample = 5

In [ ]:
## Run Inst pipeline


!ska-sdp-instrumental-calibration experimental \
--input $input_data \
--output "$pipeline_output_path" \
--stages load_data,predict_vis,ionospheric_delay,bandpass_calibraion,generate_channel_rm,export_visibilities \
--set parameters.load_data.cache_directory $cache \
--set parameters.predict_vis.eb_coeffs "/home/ska/Work/data/INST/sim/coeffs" \
--set parameters.predict_vis.lsm_csv_path "/home/ska/Work/data/INST/lg3/sky_model_cal.csv" \
--set parameters.ionospheric_delay.plot_table true \
--set parameters.ionospheric_delay.niter 50 \
--set parameters.generate_channel_rm.plot_table true \
--set parameters.generate_channel_rm.oversample $over_sample \
--set parameters.generate_channel_rm.plot_rm_config.plot_rm true \
--set parameters.export_visibilities.data_to_export "vis" \
--set parameters.export_visibilities.apply_gaintable_to_vis true \
--no-unique-output-subdir 
# --dask-scheduler $dask_schduler 